Basic imports needed:

In [4]:
from __future__ import print_function
import collections

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

tfe = tf.contrib.eager
try:
  tfe.enable_eager_execution()
except ValueError:
  pass

import matplotlib.pyplot as plt
import numpy as np